In [1]:
# An example for saving species into an RMG thermo library

In [8]:
import os
import rmgpy.data.thermo
import rmgpy.thermo

# Construct some example species with thermo

In [10]:
my_species1 = rmgpy.species.Species(smiles='CCCC')
display(my_species1)

# C4H10                   C   4H  10          G   100.000  5000.000 1147.60      1
#  6.78901843E+00 3.03598363E-02-1.21262162E-05 2.19945281E-09-1.50289520E-13    2
# -1.91057913E+04-1.17328394E+01 2.16918582E+00 3.43904442E-02-1.61558083E-06    3
# -1.30727288E-08 5.17353225E-12-1.72505136E+04 1.46546640E+01                   4
poly1 = [ 6.78901843E+00, 3.03598363E-02, -1.21262162E-05, 2.19945281E-09, -1.50289520E-13,
-1.91057913E+04,-1.17328394E+01]
poly2 = [2.16918582E+00, 3.43904442E-02,-1.61558083E-06,
-1.30727288E-08, 5.17353225E-12,-1.72505136E+04, 1.46546640E+01]
Tmin = 100
Tchange = 1147.60
Tmax = 5000

# save NasaPolynomial
my_species1.thermo = rmgpy.thermo.NASA(
    polynomials = [
        rmgpy.thermo.NASAPolynomial(coeffs=poly1, Tmin=(Tmin,'K'), Tmax=(Tchange,'K')),
        rmgpy.thermo.NASAPolynomial(coeffs=poly2, Tmin=(Tchange,'K'), Tmax=(Tmax,'K')),
    ],
    Tmin = (10,'K'),
    Tmax = (3000,'K'),
)


my_species2 = rmgpy.species.Species(smiles='CCCCO')
display(my_species2)

# save as ThermoData
my_species2.thermo = rmgpy.thermo.ThermoData(
    Tdata = ([300,400,500,600,800,1000,1500],'K'),  # this is random thermo data
    Cpdata = ([10.544,11.983,13.066,13.898,15.155,16.219,18.157],'cal/(mol*K)'),
    H298 = (54.541,'kcal/mol'),
    S298 = (48,'cal/(mol*K)'),
)


my_species = [my_species1, my_species2]

# Write the species thermo to a library

In [30]:
rmg_thermo_db = rmgpy.data.thermo.ThermoDatabase()
rmg_thermo_db.libraries['temporary_lib_name_in_memory'] = rmgpy.data.thermo.ThermoLibrary() # <---------- temporary_lib_name_in_memory is just a name in memory right now
rmg_thermo_db.libraries['temporary_lib_name_in_memory'].label = 'cool_file_name'  # <-------- this determines the name of the library.py file
rmg_thermo_db.libraries['temporary_lib_name_in_memory'].name = 'my_title'  # <------- this is the name of the library inside the file
for sp in my_species:
    entry = rmgpy.data.base.Entry()
    entry.label = str(sp)
    entry.item = sp.molecule[0]
    entry.data = sp.thermo
    rmg_thermo_db.libraries['temporary_lib_name_in_memory'].entries[entry.label] = entry

# save the results
rmg_thermo_db.save_libraries('thermo')  # <----------- this will be the name of the directory containined the libraries

In [34]:
rmg_thermo_db.libraries['temporary_lib_name_in_memory'].entries

OrderedDict([('CCCC', <Entry index=0 label="CCCC">),
             ('CCCCO', <Entry index=1 label="CCCCO">)])

# Read the thermo library back in

In [45]:
# try reading to test
# Load the new thermo library

thermo_lib = os.path.join('thermo')
rmg_thermo_db2 = rmgpy.data.thermo.ThermoDatabase()
rmg_thermo_db2.load_libraries(thermo_lib)
print(f'{len(rmg_thermo_db2.libraries["cool_file_name"].entries)} entries loaded')

# print the enthalpy at 1000 K just for the heck of it
for key in rmg_thermo_db2.libraries["cool_file_name"].entries.keys():
    thermo = rmg_thermo_db2.libraries["cool_file_name"].entries[key].data
    name = rmg_thermo_db2.libraries["cool_file_name"].entries[key].label
    print(f'{name}\tH(1000) :\t{thermo.get_enthalpy(1000):0.2f} J/mol')

2 entries loaded
CCCC	H(1000) :	-5480.42 J/mol
CCCCO	H(1000) :	269164.08 J/mol
